In [5]:
import json,csv,time
import pandas as pd
from urllib import error
from urllib.request import urlopen
from datetime import date, datetime, timedelta

In [6]:
# API of Gemini order book, see https://docs.gemini.com/rest-api/#current-order-book
# It returns 100 lines 
api_url = "https://api.gemini.com/v1/book/btcusd"
# Get data every 1 mins(60s). 
timeout = 60

In [8]:
# This function runs <=1s in my computer 
def get_data():
    try:
        #Get second start from zero
        flag = True
        while flag:
            bookdata = json.loads(urlopen(api_url).read())
            timestamp = int(bookdata["bids"][0]["timestamp"])
            remainder1 = timestamp%60
            if remainder1 < 1:
                flag = False
            else:
                #time.sleep(0.8)
                #print ('Waiting '+str(60 - remainder1%60)+'s to start next download.')
                bookdata2 = json.loads(urlopen(api_url).read())
                timestamp2 = int(bookdata["bids"][0]["timestamp"])
                remainder2 = timestamp2%60
                if remainder2 < remainder1:
                    # This line may cause some deviation of time in seconds
                    # but guranteens collecting data near 0s of this minute
                    timestamp = timestamp-remainder1
                    flag = False
                    
                    
                
        bids = bookdata["bids"]
        asks = bookdata["asks"]
        timestr = datetime.fromtimestamp(timestamp).strftime("%Y%m%d_%H%M%S")
        
        all_data_rw = bids+asks
        prices = []
        amounts = []
        for d in all_data_rw:
            prices.append(d["price"])
            amounts.append(d["amount"])
        
        df= pd.DataFrame(columns=["buy_indicator", "market_size", "price"])
        df["price"] = prices
        df["market_size"] = amounts
        
        df.loc[0:len(bids),("buy_indicator")] = 1
        df.loc[len(bids):, ("buy_indicator")] = 0
        #print (len(bids),len(asks))
        
        df.to_csv('bitcoin_'+timestr+'.txt', index=False, sep=' ', 
                    header=["buy_indicator", "market_size", "price"])
        return timestamp
        
    except error.URLError as e:
        if hasattr(e,'code'):
            print (e.code)
        if hasattr(e,'reason'):
            print (e.reason)
    except error.HTTPError as e:
        if hasattr(e,'code'):
            print(e.code)
        if hasattr(e,'reason'):
            print(e.reason)
        print('HTTPError!!!') 

In [4]:
# Execution
count = 0
while True :
    count += 1
    timestamp = get_data()
    print (str(count) + ' Data File has been downloaded!')
    time.sleep(timeout-(time.time() - timestamp+5))

Waiting 8s to start next download.
Waiting 7s to start next download.
Waiting 6s to start next download.
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
1 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
2 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
3 Data File has been downloaded!
Waiting 6s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next d

29 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
30 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
31 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
Waiting 1s to start next download.
32 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next

Waiting 1s to start next download.
58 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
Waiting 1s to start next download.
59 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
60 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
61 Data File has been downloaded!
Waiting 6s to start next download.
Waiting 5s to start next

Waiting 2s to start next download.
Waiting 1s to start next download.
86 Data File has been downloaded!
Waiting 6s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
87 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 3s to start next download.
Waiting 3s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
Waiting 1s to start next download.
88 Data File has been downloaded!
Waiting 5s to start next download.
Waiting 5s to start next download.
Waiting 4s to start next download.
Waiting 4s to start next download.
Waiting 2s to start next download.
Waiting 2s to start next download.
Waiting 1s to start next download.
89 Data File has been downloaded!
Waiting 5s to start next

KeyboardInterrupt: 